## https://redisdesktop.com/download
## http://www.runoob.com/redis/lists-lpush.html
## http://www.zendei.com/article/997.html
## https://github.com/josiahcarlson/redis-in-action/tree/master/python
## https://maoxian.de/2015/08/1342.html

### from mysql to redis

In [4]:


import redis
import MySQLdb
from collections import Counter


class dataProcessor(object):
    MYSQL_IP_ADDRESS_SERVER = 'localhost'
    MYSQL_USER = ''
    MYSQL_PASSWORD = ''
    MYSQL_DATABASE_NAME = ''

    # Redis server data
    REDIS_SERVER = 'localhost'

def sql_to_redis():
    r_redis = redis.StrictRedis(dataProcessor.REDIS_SERVER)
    print 'ok'
    database = MySQLdb.connect(dataProcessor.MYSQL_IP_ADDRESS_SERVER, dataProcessor.MYSQL_USER, dataProcessor.MYSQL_PASSWORD, dataProcessor.MYSQL_DATABASE_NAME)
    print 'ok'
    cursor = database.cursor()
    sql = 'SELECT * FROM test'
 
    cursor.execute(sql)
    data = cursor.fetchall()
    print 'ok'
    
    # # Clean redis before run again
    # # This is for test purpose
#     r_redis.delete("all_records")

#     # # Put all data from MySQL to Redis
    for row in data:
        
        # 比較字串,會有空白要注意
#         print repr(row[0].decode('utf-8'))
#         if(cmp(row[0].decode('utf-8'),u'leo ')==0):
#             print 'smae'
#         print type(row[0]),row[3]

#         #list
#         r_redis.lpush(row[0], row[3])
        
    
        #hash (好像不能是變數)
#         r_redis.hmset(row[0],{'good': row[3],'bad': row[3],})    
#         r.hmset('blogtable_userDetails_John', {'password':'abcd', 'lastlogindate':'2012-20-9'})

    # Close connection to DB and Cursor
    cursor.close()
    database.close()
    
sql_to_redis()

ok
ok
ok


### from web to redis

In [23]:
## 嘗試了一般key_value, list
REDIS_SERVER = ''
r_redis2 = redis.Redis(REDIS_SERVER)
def web2redis(url,value):
    r_redis2.incr(url,value)
#     r_redis2.lpush(url, value)

def web2redisd(url,value):
    r_redis2.decr(url,value)
web2redisd('leo1',1)
    

### from web to redis (for reback)

In [ ]:
## 嘗試了一般key_value, list
REDIS_SERVER = ''
r_redis2 = redis.Redis(REDIS_SERVER,db=1)
def web2redis(url,value):
    r_redis2.incr(url,value)
#     r_redis2.lpush(url, value)

def web2redisd(url,value):
    r_redis2.decr(url,value)

    
    
def sort_replace(row_t,test):
    # print test.keys()
    row_list=sorted(row_t.values())
    print row_list
    if (test.keys() > row_list[0]):
        row_list[0]=test
    row2=row_list
    
#     print row2
#     print row_list
    return row2

sort_replace(row,test)

web2redisd('leo1',1)

In [ ]:
##test

# row=[{},{},{},{},{}]
row=[{'a':1},{'b':4},{'c':3},{'d':6},{'e':5}]
row2=[{1:'a'},{4:'b'},{3:'c'},{6:'d'},{5:'e'}]
test={7:'f'}
test1={8:'g'}

In [22]:
import redis
REDIS_SERVER = 'localhost'
r_redis2 = redis.Redis(REDIS_SERVER)
array=[{0:''},{0:''},{0:''},{0:''},{0:''}]


def sortlovestuff(array):
    for ele in r_redis2.keys():
        row_list=sorted(array)
    #     print row_list
#         print ele
        res=int(r_redis2.get(ele))
#         print res

    #     print row_list[0].keys()[0]
        if (res > row_list[0].keys()[0]):
            row_list[0]={res:ele}
        array=row_list
        print array
    return array


sortlovestuff(array)




[{30: 'jack'}, {0: ''}, {0: ''}, {0: ''}, {0: ''}]
[{2: 'leo2'}, {0: ''}, {0: ''}, {0: ''}, {30: 'jack'}]
[{0: ''}, {0: ''}, {0: ''}, {2: 'leo2'}, {30: 'jack'}]
[{31: 'jack1'}, {0: ''}, {0: ''}, {2: 'leo2'}, {30: 'jack'}]
[{1: 'jeff'}, {0: ''}, {2: 'leo2'}, {30: 'jack'}, {31: 'jack1'}]
[{1: 'leo'}, {1: 'jeff'}, {2: 'leo2'}, {30: 'jack'}, {31: 'jack1'}]


[{1: 'leo'}, {1: 'jeff'}, {2: 'leo2'}, {30: 'jack'}, {31: 'jack1'}]

### from redis to mysql

In [78]:
REDIS_SERVER = 'localhost'
r_redis2 = redis.Redis(REDIS_SERVER)

class dataProcessor(object):
    MYSQL_IP_ADDRESS_SERVER = 'localhost'
    MYSQL_USER = 'root'
    MYSQL_PASSWORD = ''
    MYSQL_DATABASE_NAME = ''

    # Redis server data
    REDIS_SERVER = 'localhost'

database = MySQLdb.connect(dataProcessor.MYSQL_IP_ADDRESS_SERVER, dataProcessor.MYSQL_USER, dataProcessor.MYSQL_PASSWORD, dataProcessor.MYSQL_DATABASE_NAME)


##查看有幾個
for ele in r_redis2.keys():
# #     print ele
#     res=r_redis2.llen(ele)
    res=int(r_redis2.get(ele))
    cursor = database.cursor()
    ele=ele.strip()
    print ele, res
    sql = "UPDATE test2 SET age={0}  WHERE name like '{1}'".format(res,ele)
    print sql
 
    cursor.execute(sql)
    database.commit()
#     print ele,res


leo 1
UPDATE test2 SET age=1  WHERE name like 'leo'
jenny 1
UPDATE test2 SET age=1  WHERE name like 'jenny'


## transaction

In [22]:
def incre(url,valus):
    with r_redis2.pipeline() as pipe:
        while 1:
            try:
                # 关注一个key
                pipe.watch('leo2')
    #             current_value = pipe.get('leo2')
    #             next_value = int(current_value) + 1
                #事物开始
                pipe.multi()
                pipe.incr(url,valus)
                # 事务结束
                pipe.execute()
                # 把命令推送过去
                break
            except redis.WatchError:
                #如果客户端有变动的话，那么就会触发这个异常。 
                continue
            finally:
                pipe.reset()
                
def decre(url,valus):
    with r_redis2.pipeline() as pipe:
        while 1:
            try:
                # 关注一个key
                pipe.watch('leo2')
    #             current_value = pipe.get('leo2')
    #             next_value = int(current_value) + 1
                #事物开始
                pipe.multi()
                pipe.decr(url,valus)
                # 事务结束
                pipe.execute()
                # 把命令推送过去
                break
            except redis.WatchError:
                #如果客户端有变动的话，那么就会触发这个异常。 
                continue
            finally:
                pipe.reset()
                
# incre('leo2',1)
decre('leo2',1)